<a href="https://colab.research.google.com/github/smruthyunjaya05/MLE/blob/main/Healthcare_Data_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt, seaborn as sns
import warnings
warnings.filterwarnings('ignore')



In [ ]:
df = pd.read_csv("/content/Healthcare-Diabetes.csv")


In [ ]:
print("Shape:", df.shape)
print("Missing values:", df.isnull().sum())
print("Duplicate rows:", df.duplicated().sum())

if 'Id' in df.columns:
    df.drop(columns='Id', inplace=True)

display(df.head())
display(df.describe().T)

df.hist(figsize=(15, 10), bins=20)
plt.show()

sns.heatmap(df.corr(), annot=True, cmap="coolwarm")
plt.show()

sns.countplot(x='Pregnancies', data=df, hue='Outcome')
plt.show()

pairs = [
    ("Age", "BloodPressure"),
    ("Age", "Glucose"),
    ("Age", "SkinThickness"),
    ("BloodPressure", "BMI"),
    ("Insulin", "Glucose"),
    ("Pregnancies", "Glucose"),
    ("Pregnancies", "Insulin"),
    ("Pregnancies", "SkinThickness"),
    ("Pregnancies", "BMI"),
    ("Pregnancies", "BloodPressure"),
    ("Pregnancies", "DiabetesPedigreeFunction"),
    ("DiabetesPedigreeFunction", "Insulin"),
    ("DiabetesPedigreeFunction", "SkinThickness"),
    ("DiabetesPedigreeFunction", "BMI"),
    ("DiabetesPedigreeFunction", "BloodPressure"),
    ("DiabetesPedigreeFunction", "Glucose"),
    ("DiabetesPedigreeFunction", "Age")
]

# Scatter plots
for x, y in pairs:
    sns.scatterplot(data=df, x=x, y=y, hue='Outcome')
    plt.show()
